# Imports

In [ ]:
import cvxpy as cp
import numpy as np
import random
from scipy.optimize import linprog
from tqdm import tqdm
from time import time
import matplotlib.pyplot as plt

def printf(s):
  s = '\n\033[43m'+ '\033[1m' + str(s) + '\033[0m \n'
  print(s)

# Random Test case generator

In [ ]:
def create(m,n):

  if type(n) is int:
    n = [n]

  R = []; C = []
  for i in range(len(n)):
    R.append(np.random.randint(100,size=(m,n[i])))
    C.append(np.random.randint(100,size=(m,n[i])))
  
  p = [1/len(n)]*len(n)
  return R,C,p


# DOBSS and DOBSS modification 1

In [ ]:
# Stackelberg game solution -> without change
def formulate_sg(R,C,p,M=1e3,Δ=0,lp=0):

  m,n = R.shape

  z = cp.Variable((m,n), nonneg=True)
  q = cp.Variable(n,integer=True)
  a = cp.Variable(1)

  obj = p*cp.Maximize(sum(sum(cp.multiply(R,z))))
  con = [#z<=1,q<=1, #feasibility, but this is implied by the following constraints
         
         0<=q,
         sum(sum(z))==1, #sum of all xi's=1 -> belongs to probability simplex
         sum(z.T)<=1, #individual xi's <=1 -> belongs to probability simplex

         q<=sum(z), sum(z)<=1, #as zij=xi.qj
         sum(q)==1, #sum of all qj's=1 -> belongs to probability simplex

         0<=a-sum(z.T)@C, a-sum(z.T)@C-M*(1-q)<=0, #follower's problem -> maximizing follower revenue

       ]

  if lp==1:
    x = cp.Variable(m,integer=True)
    con+=[sum(z.T)==x]

  return obj, con, z, q


# Stackelberg game solution 
def formulate_sg2(R,C,p,M=1e3,Δ=0,lp=0):

  m,n = R.shape

  z = cp.Variable((m,n), nonneg=True)
  q = cp.Variable(n,integer=True)
  a = cp.Variable(1)

  obj = p*cp.Maximize(sum(sum(cp.multiply(R,z))))
  con = [
         
        #  z<=1,q<=1, #feasibility, but this is implied by the following constraints
         0<=q,
         sum(sum(z))==1, #sum of all xi's=1 -> belongs to probability simplex
         sum(z.T)<=1, #individual xi's <=1 -> belongs to probability simplex

         q<=sum(z), sum(z)<=1, #as zij=xi.qj
         sum(q)==1, #sum of all qj's=1 -> belongs to probability simplex

        #  x == sum(z.T), # by definition
        #  0<=a-x@C, a-x@C-1e3*(1-q)<=0, #follower's problem -> maximizing follower revenue
         0<=a-sum(z.T)@C, a-sum(z.T)@C-M*(1-q)<=0, #follower's problem -> maximizing follower revenue
         sum(sum(cp.multiply(R,z))) <= Δ + sum(z.T)@R + M*( sum(sum(cp.multiply(C,z))) - sum(z.T)@C ) # Select the payoff of leader which minimum among all available maximizing strategies of follower
       ]

  if lp==1:
    x = cp.Variable(m,integer=True)
    con+=[sum(z.T)==x]

  return obj, con, z, q


# Bayesian Stackelberg game solution 
def formulate_bsg(R,C,p,solver=1,Δ=0,lp=0):

  for l in range(len(R)):
    if solver==0:
      o,c,z,q = formulate_sg(R[l],C[l],p[l],Δ=Δ,lp=lp)
    elif solver==1:
      o,c,z,q = formulate_sg2(R[l],C[l],p[l],Δ=Δ,lp=lp)

    if l == 0:
      Obj = o
      Con = c
      Z = [z]
      Q = [q]
    else:
      Obj += o
      Con += c
      Z +=[z]
      Q +=[q]

  return Obj,Con,Z,Q 


# Solver of the game 
def optimal_strategy(R,C,p,show=False,solver=0,Δ=0,lp=0):

  obj, con, Z, Q = formulate_bsg(R,C,p,solver=solver,Δ=Δ,lp=lp)

  if len(Z)>1:
    c2 = []
    for l in range(len(Z)):
      if l==0:
        k = sum(Z[l].T)
      else:
        c2+=[k==sum(Z[l].T)] # additional constraint for all xi's to be equal    
    con+=c2

  prob = cp.Problem(obj, con)
  # The optimal objective value is returned by `prob.solve()`.
  result = prob.solve()

  for i in range(len(Z)):
    if i == 0:
      zf = np.sum(np.asarray(Z[i].value),axis=1)/len(Z)
    else:
      zf += np.sum(np.asarray(Z[i].value),axis=1)/len(Z)


  ans = 0
  ans2 = 0
  for i in range(len(Z)):
    ans += np.sum(Z[i].value*R[i])*p[i]
    ans2 += np.sum(Z[i].value*C[i])*p[i]


  if show==True:

    print("-------------------------------------------------")
    printf('Solver='+str(solver)+'  Δ='+str(Δ))
    print("Optimal value: ",prob.value,'\n\n')
    print("Leader\'s strategy",'\n',zf,'\n\n')
    print("Follower strategies - for different types")
    for i in range(len(Z)):
      print("Player type",i," : ", Q[i].value)
    print('\n')
    print("-------------------------------------------------")

  return ans, ans2, Z # strategy profile


# DOBSS modification 2,3

In [ ]:
def formulate_sg_ir2(R,C,p,K=1/2,L=10,frac=0.10,solver=0,lp=0):

  m,n = R.shape
  # M = max(np.max(abs(R)),np.max(abs(C)))+1e3
  M=1e2

  K2=1
  if K<=1:
    K2=K
    K=1

  z = cp.Variable((m,n), nonneg=True)
  w = cp.Variable((m,n), nonneg=True)
  q = cp.Variable(n,integer=True)
  r = cp.Variable(n,integer=True)
  g = cp.Variable(n,integer=True) 
  a = cp.Variable(1)

  con = []

  if solver!=0:
    L = cp.Variable(1)
    con+=[L==a*frac]

  obj = p*cp.Maximize(sum(sum(cp.multiply(R,w))))
  con +=[
         # Constraints from DOBSS
         sum(sum(z))==1,                                                                                     # Sum of all xi's=1     -> belongs to probability simplex
         sum(z.T)<=1,                                                                                        # Individual xi's <=1   -> belongs to probability simplex
         0<=q, sum(q)==1,                                                                                    # Sum of all qj's=1     -> belongs to probability simplex
         q<=sum(z), sum(z)<=1,                                                                               # As zij=xi.qj          -> definition of zij's
         0<=a-sum(z.T)@C, a-sum(z.T)@C-M*(1-q)<=0,                                                           #           a           -> Max follower payoff for given leader strategy

         # Constraints for modelling Irrational behaviour         
         g<=1, 0<=g,                                                                                         # Truth value variables -> binary
         sum(z.T) == sum(w.T),                                                                               # As wij=xi.q'j         -> definition of wij's
         r<=sum(w),
         0<=r, sum(r)==1,
         K2*(sum(sum(cp.multiply(C,w))) - sum(w.T)@C) >= (sum(sum(cp.multiply(R,w))) - sum(w.T)@R)/K - M*g,  # Comparisions          -> the loss of follower < k*(loss of leader) 
         a - sum(sum(cp.multiply(C,w))) <= L,                                                                # Stop loss condition   -> the loss of follower <= stop loss
         a - sum(w.T)@C >= L - M*(1-g),                                                                      # Stop loss criterion   -> truth value from g 
         a - sum(w.T)@C <= L + M*g                                                                           # Stop loss criterion   -> truth value from g 
       ]


  if lp==1:
    x = cp.Variable(m,integer=True)
    con+=[sum(z.T)==x]
    con+=[sum(w.T)==x]

  return obj, con, w, z, q, g, a, r



# Bayesian Stackelberg game solution 
def formulate_bsg2(R,C,p,solver=0,K=2,L=10,frac=0.1,lp=0):

  for l in range(len(R)):

    o, c, w, z, q, g, a, r = formulate_sg_ir2(R[l],C[l],p[l],K=K,L=L,solver=solver,frac=frac,lp=lp)    

    if l == 0:
      Obj = o
      Con = c
      W = [w]
      Z = [z]
      Q = [q]
      G = [g]
      A = [a]
      Rr = [r]

    else:
      Obj += o
      Con += c
      W += [w]
      Z += [z]
      Q += [q]
      G += [g]
      A += [a]
      Rr += [r]

  return Obj,Con,W,Z,Q,G,A,Rr


def optimal_strategy2(R,C,p,show=False,K=2,L=10,solver=0,frac=0,lp=0):

  Obj,Con,W,Z,Q,G,A,Rr = formulate_bsg2(R,C,p,K=K,L=L,solver=solver,frac=frac,lp=lp)

  if len(W)>1:
    c2 = []
    for l in range(len(W)):
      if l==0:
        k = sum(W[l].T)
      else:
        c2+=[k==sum(W[l].T)] # additional constraint for all xi's to be equal    
    Con+=c2

  prob = cp.Problem(Obj,Con)
  result = prob.solve()

  ans = 0
  ans2 = 0
  for i in range(len(W)):
    ans += np.sum(W[i].value*R[i])*p[i]
    ans2 += np.sum(W[i].value*C[i])*p[i]

  for i in range(len(W)):
    if i == 0:
      wf = np.sum(np.asarray(W[i].value),axis=1)
    else:
      wf += np.sum(np.asarray(W[i].value),axis=1)
  wf = wf/len(W)


  if show == True:

    if solver == 0:
      printf('Solver='+str(solver+3)+'  K-factor='+str(K) +';  Stop loss ='+str(L))
    else:
      printf('Solver='+str(solver+3)+'  K-factor='+str(K) +';  Fractional loss ='+str(frac))

    print("-------------------------------------------------")
    print("Objective value "+str(ans))

    print("-------------------------------------------------")
    print("W")
    for i in range(len(W)):
      print("player type ",i+1)
      print(W[i].value)

    print("-------------------------------------------------")
    print("Z")
    for i in range(len(Z)):
      print("player type ",i+1)
      print(Z[i].value)
    
    print("-------------------------------------------------")
    print("Q")
    for i in range(len(Q)):
      print("player type ",i+1)
      print(Q[i].value)

    print("-------------------------------------------------")
    print("G")
    for i in range(len(G)):
      print("player type ",i+1)
      print(G[i].value)
    
    print("-------------------------------------------------")
    print("A")
    for i in range(len(A)):
      print("player type ",i+1)
      print(A[i].value)

    print("-------------------------------------------------")
    print("Rr")
    for i in range(len(Rr)):
      print("player type ",i+1)
      print(Rr[i].value)

    print("-------------------------------------------------")

    print("Leader payoff: ",ans,'\n\n')
    
    print("Follower payoff - for different types")
    for i in range(len(W)):
      print("Player type",i," : ", np.sum(W[i].value*C[i]))
    print('\n')

    print("-------------------------------------------------")

    print("Leader\'s strategy",'\n',wf,'\n\n')

    print("Follower strategies - for different types")
    L2 = []
    for i in range(len(W)):
      print("Player type",i," : ", np.sum(np.asarray(W[i].value),axis=0))
      L2.append(np.sum(np.asarray(W[i].value),axis=0))
    print('\n')
    
    for i in range(len(W)):
      print("W matrix",i," : ", np.asarray(W[i].value))
    print('\n')

    print("-------------------------------------------------")


  return ans, ans2, W 
  

# Comparing solutions


# 4


## Testing the effect of the parameter `L` in choosing a mixed strategy.

Here is a simple example where we see a mixed strategy is chosen by the algorithm as an output which shows some characteristics of risk avoidance.

Here, we see how we pick a mixed strategy based on the amount of loss that is being dictated by the stop loss parameter `L`. (Here, we would like to limit the effect of choice with respect to the parameter `K`, so `K` is chosen to be small and the selected parameter is unaffected by `K`)

The example is as follows:

|      | G1 | G2 |
|:---  |:---:|:---:|
|**B1**| 4,5 | 6,5 |
|**B2**| 5,4 | 5,5 |


There is a risky strategy `B1` for the leader and another strategy `B2`, which is not very risky.
The follower has `G1` and `G2`, where `G2` is his stable strategy and `G2` is when he wants to take loss to incur damage to the leader. Now the leader has to mix his strategies in order to make the follower believe the loss faced by it will exceed it's stop loss if it choses the damage incurring strategy.

The results also reflect the same, with an increase in value of `L`, the strategy chosen is more conservative and the probability for picking the riskier strategy is becoming lesser and lesser. The payoff is also becoming lower as the risk is lowered.

In [ ]:
R = [np.asarray ( [[ 4, 6 ],
                   [ 5, 5 ]
                   ] ) ]
# print(R)

C = [np.asarray ( [[ 5, 5 ],
                   [ 4, 5 ]
                   ] ) ]
# print(C)

p = [1]


print("---------------------------------------------\n")
print("Original DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
v1,f1,Z1 = optimal_strategy(R,C,p,show=False,solver=0,lp=0)
print( "{:<15}".format("Original DOBSS") ,"{:<10} {:1} {:<10}".format(  round(v1,2),'|',round(f1,2)))

print("---------------------------------------------\n")
v2,f2,Z2 = optimal_strategy(R,C,p,show=False,solver=1,Δ=0,lp=0)
print("Mod-1 DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
print( "{:<15}".format("Mod-1 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v2,2),'|',round(f2,2)))


print("---------------------------------------------\n")

print("Mod-2 DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.1,L=0.9,solver=0,lp=0)
print( "{:<15}".format("K=0.1,L=1.1")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))

v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.1,L=0.5,solver=0,lp=0)
print( "{:<15}".format("K=0.1,L=0.5")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))

v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.1,L=0.1,solver=0,lp=0)
print( "{:<15}".format("K=0.1,L=0.3")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))


---------------------------------------------

Original DOBSS
                Leader     | Follower  
Original DOBSS  6.0        | 5.0       
---------------------------------------------

Mod-1 DOBSS
                Leader     | Follower  
Mod-1 DOBSS     6.0        | 5.0       
---------------------------------------------

Mod-2 DOBSS
                Leader     | Follower  
K=0.1,L=1.1     5.1        | 5.0       
K=0.1,L=0.5     5.5        | 5.0       
K=0.1,L=0.3     5.9        | 5.0       


## Testing the effect of the parameter `K` in choosing a mixed strategy.

Here, we see how we pick a mixed strategy based on the  that is being dictated by the stop loss parameter `L`. (Here, we would like to limit the effect of choice with respect to the parameter `K`, so `K` is chosen to be small and the selected parameter is unaffected by `K`)

The example is as follows:

|      | G1 | G2 |
|:---  |:---:|:---:|
|**B1**| 1,1 | 0,0 |
|**B2**| 0,1 | 0,0 |


B1,G1 is the only pair strategy here which gives the leader a non zero payoff.
G1 gives the follower a guaranteed payoff of 1 and G2 gives 0. So, if the follower is willing to take a loss, then he will choose G2. But he will only take a loss if the leader is losing some sufficient amount of payoff.

The leader has to make it look less appealing for the follower to choose G2 by mixing B1, B2 with some probabilities such that the follower will not want to take the loss for the sake of making the leader lose some insufficient amount of payoff.

In the following test runs, we see that:-
* When the follower has a K value of `x`, follower is not going to sacrifice it's utility for the sake of reducing the payoff of follower by any amount less than K times it's loss. 
* So, the leader can choose the mixing probabilities so as to make the follower not choose the irrational strategy to lower his payoff.

The results also reflect the same, an increase in value of `K` indicates that the follower is not very easily giving up his payoff. So, the estimates payoffs are higher in the hope that follower doesn't deviate much from perfect rationality.

With decrease in `k` value, we are giving a more conservative estimate, hence follower tends to choose the `taking a hit on self payoff` strategy to make the other option look little less appealing. So, in a way the algorithm would output a follower strategy which is not very appealing to deviate from for the follower and hence it adds stability for the picked strategies.


In [ ]:
R = [np.asarray ( [[ 1, 0 ],
                   [ 0, 0 ]
                   ] ) ]
# print(R)

C = [np.asarray ( [[ 1, 0 ],
                   [ 1, 0 ]
                   ] ) ]
# print(C)

p = [1]


print("---------------------------------------------\n")
print("Original DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
v1,f1,Z1 = optimal_strategy(R,C,p,show=False,solver=0,lp=0)
print( "{:<15}".format("Original DOBSS") ,"{:<10} {:1} {:<10}".format(  round(v1,2),'|',round(f1,2)))

print("---------------------------------------------\n")
v2,f2,Z2 = optimal_strategy(R,C,p,show=False,solver=1,Δ=0,lp=0)
print("Mod-1 DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
print( "{:<15}".format("Mod-1 DOBSS")    ,"{:<10} {:1} {:<10}".format(  round(v2,2),'|',round(f2,2)))


print("---------------------------------------------\n")

print("Mod-2 DOBSS")
print(  "{:<15}".format(" "), "{:<10} {:1} {:<10}".format(  "Leader",'|', "Follower"  ))
v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.1,L=1.1,solver=0,lp=0)
print( "{:<15}".format("K=0.1,L=1.1")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))

v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.5,L=1.1,solver=0,lp=0)
print( "{:<15}".format("K=0.5,L=1.1")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))

v3,f3,Z = optimal_strategy2(R,C,p,show=False,K=0.9,L=1.1,solver=0,lp=0)
print( "{:<15}".format("K=0.9,L=1.1")    ,"{:<10} {:1} {:<10}".format(  round(v3,2),'|',round(f3,2)))


---------------------------------------------

Original DOBSS
                Leader     | Follower  
Original DOBSS  1.0        | 1.0       
---------------------------------------------

Mod-1 DOBSS
                Leader     | Follower  
Mod-1 DOBSS     1.0        | 1.0       
---------------------------------------------

Mod-2 DOBSS
                Leader     | Follower  
K=0.1,L=1.1     0.1        | 1.0       
K=0.5,L=1.1     0.5        | 1.0       
K=0.9,L=1.1     0.9        | 1.0       


## Endnote

These two examples were chosen for simplification of the problem and easier understanding. The algorithm works in line with the expectations of the given inputs.

The cases were taken individually for checking the behaviour of the output strategies with respect to individual parameters as suitable examples also are hard to construct that can give us intuitively understandable changes in the output strategies when both parameters are altered.

Except for a few precision errors, no other error was encountered and the output strategies are according to our expectations from the algorithm.